In [14]:
import numpy as np

In [15]:
commande = {
    # switch
        # mode vol / grappin
    'state_switch_1': ..., # 0 --> position haut du switch 1 / 1 --> position basse du switch 1
    'state_switch_2': ..., # 0 --> position haut du switch 2 / 1 --> position basse du switch 2
    'state_switch_3': ..., # 0 --> position haut du switch 3 / 1 --> position milieu du switch 3 / 2 --> position basse du switch 3
    'state_switch_4': ..., # 0 --> position haut du switch 4 / 1 --> position basse du switch 4

    # position slider 
    'slider': ..., # valeur entre 0 et 100

    # position joystick
    'joystick': [..., ...], # [channel_1, channel_2]

    # potentiomètre 1
    'potentiometre_1': ..., # modification de la vitesse de rotation des moteurs grappin
}

In [16]:
def map_linear(x, in_min, in_max, out_min, out_max):
    return int((x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min)


In [17]:
# retour du joystick pour direction

def GetCommMotors(ch1, ch2):

    DEAD_ZONE = 0.2    # 0 <= x <= 1

    r = abs(ch1) + abs(ch2)
    r_mapped = map_linear(r, DEAD_ZONE, 1, 0, 1)

    if r <= DEAD_ZONE:
        ch1_ = 0
        ch2_ = 0
    elif r >= DEAD_ZONE and r <= 1:
        ch1_ = ch1 * r_mapped
        ch2_ = ch2 * r_mapped
    elif r >= 1:
        ch1_ = ch1/r
        ch2_ = ch2/r

    Fv_G = -ch1_
    Fv_D = ch1_
    Fp_G = ch2_
    Fp_D = ch2_

    F_G = Fv_G + Fp_G
    F_D = Fv_D + Fp_D

    if F_G < 0:
        angle_G = 0
    elif F_G == 0:
        angle_G = 90
    elif F_G > 0:
        angle_G = 180

    if F_D < 0:
        angle_D = 0
    elif F_D == 0:
        angle_D = 90
    elif F_D > 0:
        angle_D = 180

    return abs(F_G), abs(F_D), angle_G, angle_D

In [18]:
def GetSpeedGrappin(ch3):
    
    DEAD_ZONE = 0.2     # 0 <= x <= 2
    MAX_SPEED = 1000
    
    if ch3 < -DEAD_ZONE/2:
        speedGrappin = map_linear(ch3, -1, -DEAD_ZONE/2, -MAX_SPEED, 0)

    elif (-DEAD_ZONE/2 <= ch3 and ch3 <= DEAD_ZONE/2):
        speedGrappin = 0

    elif (DEAD_ZONE/2 < ch3):
        speedGrappin = map_linear(ch3, DEAD_ZONE/2, 1, 0, MAX_SPEED)
    
    return speedGrappin

In [19]:
def FlightControl(commande) -> dict:

    speed_motor_G, speed_motor_D, angle_servomot_G, angle_servomot_D = GetCommMotors(commande['joystick'][0], commande['joystick'][1])

    instruction_motor = {

        # gestion moteur avant droit
        'speed_AvD': speed_motor_D,
        'angle_AvD': angle_servomot_D,

        # moteur avant gauche
        'speed_AvG': speed_motor_G,
        'angle_AvG': angle_servomot_G,

        # moteur arrière droit
        'speed_ArD': speed_motor_D,
        'angle_ArD': angle_servomot_D,

        # moteur arrière gauche
        'speed_ArG': speed_motor_G,
        'angle_ArG': angle_servomot_G

    }

    speed_stepmotor = GetSpeedGrappin(commande['slider']) # valeur entre -1 et 1

    instruction_grappin = {
        # control des vitesses de rotation des moteurs pas à pas
        'speed_stepmotor' : speed_stepmotor
    }

    control = {
        'instruction_motor': instruction_motor,
        'instruction_grappin': instruction_grappin,
        'instruction_ballast': ...,
    }


    return control